# Tutorial 03: Running RLlib Experiments

This tutorial walks you through the process of running traffic simulations in Flow with trainable RLlib-powered agents. Autonomous agents will learn to maximize a certain reward over the rollouts, using the [**RLlib**](https://ray.readthedocs.io/en/latest/rllib.html) library ([citation](https://arxiv.org/abs/1712.09381)) ([installation instructions](https://flow.readthedocs.io/en/latest/flow_setup.html#optional-install-ray-rllib)). Simulations of this form will depict the propensity of RL agents to influence the traffic of a human fleet in order to make the whole fleet more efficient (for some given metrics). 

In this tutorial, we simulate an initially perturbed single lane ring road, where we introduce a single autonomous vehicle. We witness that, after some training, that the autonomous vehicle learns to dissipate the formation and propagation of "phantom jams" which form when only human driver dynamics are involved.

## 1. Components of a Simulation
All simulations, both in the presence and absence of RL, require two components: a *network*, and an *environment*. Networks describe the features of the transportation network used in simulation. This includes the positions and properties of nodes and edges constituting the lanes and junctions, as well as properties of the vehicles, traffic lights, inflows, etc... in the network. Environments, on the other hand, initialize, reset, and advance simulations, and act as the primary interface between the reinforcement learning algorithm and the network. Moreover, custom environments may be used to modify the dynamical features of an network. Finally, in the RL case, it is in the *environment* that the state/action spaces and the reward function are defined. 

## 2. Setting up a Network
Flow contains a plethora of pre-designed networks used to replicate highways, intersections, and merges in both closed and open settings. All these networks are located in flow/networks. For this tutorial, which involves a single lane ring road, we will use the network `RingNetwork`.

### 2.1 Setting up Network Parameters

The network mentioned at the start of this section, as well as all other networks in Flow, are parameterized by the following arguments: 
* name
* vehicles
* net_params
* initial_config

These parameters are explained in detail in `tutorial01_sumo.ipynb`. Moreover, all parameters excluding vehicles (covered in section 2.2) do not change from the previous tutorial. Accordingly, we specify them nearly as we have before, and leave further explanations of the parameters to `tutorial01_sumo.ipynb`.

We begin by choosing the network the experiment will be trained on. We use one of Flow's builtin networks, located in `flow.networks`. A list of all available networks can be found by running the script below.

In [1]:
import flow.networks as networks

# print(networks.__all__)

In this tutorial, we choose to use the ring road network. The network class is then:

In [2]:
from flow.networks import RingNetwork

# ring road network class
network_name = RingNetwork

One key difference between SUMO and RLlib experiments is that, in RLlib experiments, the network classes do not need to be defined; instead users should simply name the network class they wish to use. Later on, an environment setup module will import the correct network class based on the provided names.

In [3]:
# input parameter classes to the network class
from flow.core.params import NetParams, InitialConfig

# name of the network
name = "training_example15"

# network-specific parameters
from flow.networks.ring import ADDITIONAL_NET_PARAMS
net_params = NetParams(additional_params=ADDITIONAL_NET_PARAMS)

# initial configuration to vehicles
initial_config = InitialConfig(spacing="uniform", perturbation=1)

### 2.2 Adding Trainable Autonomous Vehicles
The `Vehicles` class stores state information on all vehicles in the network. This class is used to identify the dynamical features of a vehicle and whether it is controlled by a reinforcement learning agent. Morover, information pertaining to the observations and reward function can be collected from various `get` methods within this class.

The dynamics of vehicles in the `Vehicles` class can either be depicted by sumo or by the dynamical methods located in flow/controllers. For human-driven vehicles, we use the IDM model for acceleration behavior, with exogenous gaussian acceleration noise with std 0.2 m/s2 to induce perturbations that produce stop-and-go behavior. In addition, we use the `ContinousRouter` routing controller so that the vehicles may maintain their routes closed networks.

As we have done in `tutorial01_sumo.ipynb`, human-driven vehicles are defined in the `VehicleParams` class as follows:

In [4]:
# vehicles class
from flow.core.params import VehicleParams

# vehicles dynamics models
from flow.controllers import IDMController, ContinuousRouter

vehicles = VehicleParams()
#vehicles.add("human",
#             acceleration_controller=(IDMController, {}),
#             routing_controller=(ContinuousRouter, {}),
#             num_vehicles=10)

The above addition to the `Vehicles` class only accounts for 21 of the 22 vehicles that are placed in the network. We now add an additional trainable autuonomous vehicle whose actions are dictated by an RL agent. This is done by specifying an `RLController` as the acceleraton controller to the vehicle. 

In [5]:
from flow.controllers import RLController

Note that this controller serves primarirly as a placeholder that marks the vehicle as a component of the RL agent, meaning that lane changing and routing actions can also be specified by the RL agent to this vehicle.

We finally add the vehicle as follows, while again using the `ContinuousRouter` to perpetually maintain the vehicle within the network.

In [6]:
# from flow.energy_models.toyota_energy import TacomaEnergy
# vehicles.add(veh_id="rl",
#              acceleration_controller=(RLController, {}),
#              routing_controller=(ContinuousRouter, {}),
#              initial_speed =20,
#              energy_model = TacomaEnergy,
#              num_vehicles=1)


vehicles.add(veh_id="rl",
             acceleration_controller=(RLController, {}),
             routing_controller=(ContinuousRouter, {}),
             initial_speed =20,
             num_vehicles=1)

## 3. Setting up an Environment

Several environments in Flow exist to train RL agents of different forms (e.g. autonomous vehicles, traffic lights) to perform a variety of different tasks. The use of an environment allows us to view the cumulative reward simulation rollouts receive, along with to specify the state/action spaces.

Sumo envrionments in Flow are parametrized by three components:
* `SumoParams`
* `EnvParams`
* `Network`

### 3.1 SumoParams
`SumoParams` specifies simulation-specific variables. These variables include the length of any simulation step and whether to render the GUI when running the experiment. For this example, we consider a simulation step length of 0.1s and deactivate the GUI. 

**Note** For training purposes, it is highly recommanded to deactivate the GUI in order to avoid global slow down. In such case, one just needs to specify the following: `render=False`

In [7]:
from flow.core.params import SumoParams

sim_params = SumoParams(sim_step=0.1, render=False)

### 3.2 EnvParams

`EnvParams` specifies environment and experiment-specific parameters that either affect the training process or the dynamics of various components within the network. For the environment `WaveAttenuationPOEnv`, these parameters are used to dictate bounds on the accelerations of the autonomous vehicles, as well as the range of ring lengths (and accordingly network densities) the agent is trained on.

Finally, it is important to specify here the *horizon* of the experiment, which is the duration of one episode (during which the RL-agent acquire data). 

In [8]:
from flow.core.params import EnvParams

# Define horizon as a variable to ensure consistent use across notebook
HORIZON=1000

env_params = EnvParams(
    # length of one rollout
    horizon=HORIZON,

    additional_params={
        # maximum acceleration of autonomous vehicles
        "max_accel": 1,
        # maximum deceleration of autonomous vehicles
        "max_decel": 1,
        # bounds on the ranges of ring road lengths the autonomous vehicle 
        # is trained on
        "ring_length": [220, 270],
    },
)

### 3.3 Initializing a Gym Environment

Now, we have to specify our Gym Environment and the algorithm that our RL agents will use. Similar to the network, we choose to use on of Flow's builtin environments, a list of which is provided by the script below.

In [9]:
import flow.envs as flowenvs

print(flowenvs.__all__)

['Env', 'AccelEnv', 'LaneChangeAccelEnv', 'LaneChangeAccelPOEnv', 'TrafficLightGridTestEnv', 'MergePOEnv', 'BottleneckEnv', 'BottleneckAccelEnv', 'WaveAttenuationEnv', 'WaveAttenuationPOEnv', 'EnergyOptEnv', 'EnergyOptPOEnv', 'TrafficLightGridEnv', 'TrafficLightGridPOEnv', 'TrafficLightGridBenchmarkEnv', 'BottleneckDesiredVelocityEnv', 'TestEnv', 'BayBridgeEnv', 'SingleStraightRoad', 'BottleNeckAccelEnv', 'DesiredVelocityEnv', 'PO_TrafficLightGridEnv', 'GreenWaveTestEnv']


We will use the environment "WaveAttenuationPOEnv", which is used to train autonomous vehicles to attenuate the formation and propagation of waves in a partially observable variable density ring road. To create the Gym Environment, the only necessary parameters are the environment name plus the previously defined variables. These are defined as follows:

In [10]:
from flow.envs import EnergyOptPOEnv

env_name = EnergyOptPOEnv

In [11]:
# from flow.envs import WaveAttenuationPOEnv

# env_name = WaveAttenuationPOEnv

### 3.4 Setting up Flow Parameters

RLlib experiments both generate a `params.json` file for each experiment run. For RLlib experiments, the parameters defining the Flow network and environment must be stored as well. As such, in this section we define the dictionary `flow_params`, which contains the variables required by the utility function `make_create_env`. `make_create_env` is a higher-order function which returns a function `create_env` that initializes a Gym environment corresponding to the Flow network specified.

In [12]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict(
    # name of the experiment
    exp_tag=name,
    # name of the flow environment the experiment is running on
    env_name=env_name,
    # name of the network class the experiment uses
    network=network_name,
    # simulator that is used by the experiment
    simulator='traci',
    # simulation-related parameters
    sim=sim_params,
    # environment related parameters (see flow.core.params.EnvParams)
    env=env_params,
    # network-related parameters (see flow.core.params.NetParams and
    # the network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,
    # vehicles to be placed in the network at the start of a rollout 
    # (see flow.core.vehicles.Vehicles)
    veh=vehicles,
    # (optional) parameters affecting the positioning of vehicles upon 
    # initialization/reset (see flow.core.params.InitialConfig)
    initial=initial_config
)

## 4 Running RL experiments in Ray

### 4.1 Import 

First, we must import modules required to run experiments in Ray. The `json` package is required to store the Flow experiment parameters in the `params.json` file, as is `FlowParamsEncoder`. Ray-related imports are required: the PPO algorithm agent, `ray.tune`'s experiment runner, and environment helper methods `register_env` and `make_create_env`.

In [13]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
# from ray.rllib.agents.agent import get_agent_class
#from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

Instructions for updating:
non-resource variables are not supported in the long term


### 4.2 Initializing Ray
Here, we initialize Ray and experiment-based constant variables specifying parallelism in the experiment as well as experiment batch size in terms of number of rollouts.

In [14]:
# number of parallel workers
N_CPUS = 4
# number of rollouts per training iteration
N_ROLLOUTS = 1
#ray.shutdown()
ray.init(num_cpus=N_CPUS)

2020-07-23 18:17:23,170	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-07-23_18-17-23_169420_16993/logs.
2020-07-23 18:17:23,287	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:41576 to respond...
2020-07-23 18:17:23,436	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:17355 to respond...
2020-07-23 18:17:23,442	INFO services.py:809 -- Starting Redis shard with 3.3 GB max memory.
2020-07-23 18:17:23,518	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-07-23_18-17-23_169420_16993/logs.
2020-07-23 18:17:23,524	WARNING services.py:1330 -- WARNING: The default object store size of 4.96 GB will use more than 50% of the available memory on this node (8.74 GB). Consider setting the object store memory manually to a smaller size to avoid memory contention with other applications.
2020-07-23 18:17:23,527	INFO services.py:1475 -- Starting the Plasma object store with 4.96 GB memory us

{'node_ip_address': '192.168.100.38',
 'redis_address': '192.168.100.38:41576',
 'object_store_address': '/tmp/ray/session_2020-07-23_18-17-23_169420_16993/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-07-23_18-17-23_169420_16993/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-07-23_18-17-23_169420_16993'}

### 4.3 Configuration and Setup
Here, we copy and modify the default configuration for the [PPO algorithm](https://arxiv.org/abs/1707.06347). The agent has the number of parallel workers specified, a batch size corresponding to `N_ROLLOUTS` rollouts (each of which has length `HORIZON` steps), a discount rate $\gamma$ of 0.999, two hidden layers of size 16, uses Generalized Advantage Estimation, $\lambda$ of 0.97, and other parameters as set below.

Once `config` contains the desired parameters, a JSON string corresponding to the `flow_params` specified in section 3 is generated. The `FlowParamsEncoder` maps objects to string representations so that the experiment can be reproduced later. That string representation is stored within the `env_config` section of the `config` dictionary. Later, `config` is written out to the file `params.json`. 

Next, we call `make_create_env` and pass in the `flow_params` to return a function we can use to register our Flow environment with Gym. 

In [15]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS - 1  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate
config["model"].update({"fcnet_hiddens": [16, 16]})  # size of hidden layers in network
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

### 4.4 Running Experiments

Here, we use the `run_experiments` function from `ray.tune`. The function takes a dictionary with one key, a name corresponding to the experiment, and one value, itself a dictionary containing parameters for training.

In [16]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 20,  # number of iterations between checkpoints
        "checkpoint_at_end": True,  # generate a checkpoint at the end
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 700,  # number of iterations to stop after
        },
    },
})

2020-07-23 18:17:23,862	INFO trial_runner.py:176 -- Starting a new experiment.
2020-07-23 18:17:23,988	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.
2020-07-23 18:17:24,012	WARNING logger.py:227 -- Could not instantiate <class 'ray.tune.logger.TFLogger'> - skipping.
2020-07-23 18:17:24,014	ERROR log_sync.py:34 -- Log sync requires cluster to be setup with `ray up`.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/4 CPUs, 0/0 GPUs
Memory usage on this node: 7.8/16.5 GB



2020-07-23 18:17:24,101	WARNING util.py:145 -- The `start_trial` operation took 0.11872601509094238 seconds to complete, which may be a performance bottleneck.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 4/4 CPUs, 0/0 GPUs
Memory usage on this node: 7.9/16.5 GB
Result logdir: /home/solom/ray_results/training_example15
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - PPO_EnergyOptPOEnv-v0_0:	RUNNING

(pid=17036) WARNING:tensorflow:From /home/solom/anaconda3/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=17036) Instructions for updating:
(pid=17036) non-resource variables are not supported in the long term
(pid=17036) 2020-07-23 18:17:27,659	WARNING ppo.py:143 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
(pid=17036) 2020-07-23 18:17:28,824	INFO rollout_worker.py:319 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
(pid=17036) 2020-07-23 18:17:28.826642: I tensorflow

(pid=17034) 2020-07-23 18:17:34,999	INFO rollout_worker.py:319 -- Creating policy evaluation worker 2 on CPU (please ignore any CUDA init errors)
(pid=17034) 2020-07-23 18:17:35.002078: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=17034) 2020-07-23 18:17:35.034957: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 1999965000 Hz
(pid=17034) 2020-07-23 18:17:35.035318: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f22b8000b20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
(pid=17034) 2020-07-23 18:17:35.035370: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
(pid=17034) 2020-07-23 18:17:35.039243: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared o

...

(pid=17035) 
(pid=17035) -----------------------
(pid=17035) ring length: 257
(pid=17035) -----------------------
(pid=17033) 
(pid=17033) -----------------------
(pid=17033) ring length: 229
(pid=17033) -----------------------
(pid=17034) 
(pid=17034) -----------------------
(pid=17034) ring length: 265
(pid=17034) -----------------------
Result for PPO_EnergyOptPOEnv-v0_0:
  custom_metrics: {}
  date: 2020-07-23_18-40-32
  done: false
  episode_len_mean: 1000.0
  episode_reward_max: -5.468003427410517
  episode_reward_mean: -626.5181867641603
  episode_reward_min: -14525.788691568561
  episodes_this_iter: 0
  episodes_total: 651
  experiment_id: 7eca8b5e43da4c948c7b8f10416b7db0
  hostname: solom-XPS-13-9380
  info:
    grad_time_ms: 12.658
    learner:
      default_policy:
        cur_kl_coeff: 0.0
        cur_lr: 4.999999873689376e-05
        entropy: 1.362951397895813
        entropy_coeff: 0.0
        kl: 1.1693834949255688e-06
        policy_loss: -3.139972614008002e-05
        

(pid=17033) 
(pid=17033) -----------------------
(pid=17033) ring length: 233
(pid=17033) -----------------------
(pid=17035) 
(pid=17035) -----------------------
(pid=17035) ring length: 270
(pid=17035) -----------------------
(pid=17034) 
(pid=17034) -----------------------
(pid=17034) ring length: 257
(pid=17034) -----------------------
Result for PPO_EnergyOptPOEnv-v0_0:
  custom_metrics: {}
  date: 2020-07-23_18-40-53
  done: false
  episode_len_mean: 1000.0
  episode_reward_max: -5.604987030729615
  episode_reward_mean: -639.3319648990032
  episode_reward_min: -14525.788691568561
  episodes_this_iter: 0
  episodes_total: 663
  experiment_id: 7eca8b5e43da4c948c7b8f10416b7db0
  hostname: solom-XPS-13-9380
  info:
    grad_time_ms: 14.613
    learner:
      default_policy:
        cur_kl_coeff: 0.0
        cur_lr: 4.999999873689376e-05
        entropy: 1.3421895503997803
        entropy_coeff: 0.0
        kl: 3.0023396902834065e-05
        policy_loss: -0.00042773628956638277
      

(pid=17035) 
(pid=17035) -----------------------
(pid=17035) ring length: 248
(pid=17035) -----------------------
(pid=17034) 
(pid=17034) -----------------------
(pid=17034) ring length: 251
(pid=17034) -----------------------
(pid=17033) 
(pid=17033) -----------------------
(pid=17033) ring length: 262
(pid=17033) -----------------------
Result for PPO_EnergyOptPOEnv-v0_0:
  custom_metrics: {}
  date: 2020-07-23_18-41-16
  done: false
  episode_len_mean: 1000.0
  episode_reward_max: -5.604987030729615
  episode_reward_mean: -816.2363555732586
  episode_reward_min: -15803.393034681561
  episodes_this_iter: 0
  episodes_total: 675
  experiment_id: 7eca8b5e43da4c948c7b8f10416b7db0
  hostname: solom-XPS-13-9380
  info:
    grad_time_ms: 15.731
    learner:
      default_policy:
        cur_kl_coeff: 0.0
        cur_lr: 4.999999873689376e-05
        entropy: 1.350881576538086
        entropy_coeff: 0.0
        kl: 5.599856223170718e-08
        policy_loss: 8.45336944621522e-06
        tot

(pid=17033) 
(pid=17033) -----------------------
(pid=17033) ring length: 265
(pid=17033) -----------------------
(pid=17034) 
(pid=17034) -----------------------
(pid=17034) ring length: 229
(pid=17034) -----------------------
(pid=17035) 
(pid=17035) -----------------------
(pid=17035) ring length: 225
(pid=17035) -----------------------
Result for PPO_EnergyOptPOEnv-v0_0:
  custom_metrics: {}
  date: 2020-07-23_18-41-38
  done: false
  episode_len_mean: 1000.0
  episode_reward_max: -5.94247897100012
  episode_reward_mean: -784.7040059759457
  episode_reward_min: -15803.393034681561
  episodes_this_iter: 0
  episodes_total: 687
  experiment_id: 7eca8b5e43da4c948c7b8f10416b7db0
  hostname: solom-XPS-13-9380
  info:
    grad_time_ms: 12.983
    learner:
      default_policy:
        cur_kl_coeff: 0.0
        cur_lr: 4.999999873689376e-05
        entropy: 1.3487855195999146
        entropy_coeff: 0.0
        kl: 6.465912065323209e-06
        policy_loss: -0.00022122669906821102
        

(pid=17035) 
(pid=17035) -----------------------
(pid=17035) ring length: 239
(pid=17035) -----------------------
(pid=17033) 
(pid=17033) -----------------------
(pid=17033) ring length: 247
(pid=17033) -----------------------
(pid=17034) 
(pid=17034) -----------------------
(pid=17034) ring length: 223
(pid=17034) -----------------------
Result for PPO_EnergyOptPOEnv-v0_0:
  custom_metrics: {}
  date: 2020-07-23_18-41-58
  done: true
  episode_len_mean: 1000.0
  episode_reward_max: -6.010638117001379
  episode_reward_mean: -702.7642679412602
  episode_reward_min: -15803.393034681561
  episodes_this_iter: 0
  episodes_total: 699
  experiment_id: 7eca8b5e43da4c948c7b8f10416b7db0
  hostname: solom-XPS-13-9380
  info:
    grad_time_ms: 13.586
    learner:
      default_policy:
        cur_kl_coeff: 0.0
        cur_lr: 4.999999873689376e-05
        entropy: 1.0918323993682861
        entropy_coeff: 0.0
        kl: 3.411716170376167e-05
        policy_loss: -0.0003285584389232099
        t

### 4.5 Visualizing the results

The simulation results are saved within the `ray_results/training_example` directory (we defined `training_example` at the start of this tutorial). The `ray_results` folder is by default located at your root `~/ray_results`. 

You can run `tensorboard --logdir=~/ray_results/training_example` (install it with `pip install tensorboard`) to visualize the different data outputted by your simulation.

For more instructions about visualizing, please see `tutorial05_visualize.ipynb`. 

### 4.6 Restart from a checkpoint / Transfer learning

If you wish to do transfer learning, or to resume a previous training, you will need to start the simulation from a previous checkpoint. To do that, you can add a `restore` parameter in the `run_experiments` argument, as follows:

```python
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "restore": "/ray_results/experiment/dir/checkpoint_50/checkpoint-50"
        "checkpoint_freq": 1,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 1,
        },
    },
})
```

The `"restore"` path should be such that the `[restore]/.tune_metadata` file exists.

There is also a `"resume"` parameter that you can set to `True` if you just wish to continue the training from a previously saved checkpoint, in case you are still training on the same experiment. 

In [17]:
# trials = run_experiments({
#     flow_params["exp_tag"]: {
#         "run": alg_run,
#         "env": gym_name,
#         "config": {
#             **config
#         },
#         "restore": "/ray_results/training_example13/PPO_EnergyOptPOEnv-v0_0_2020-07-23_13-30-07yze28sum/checkpoint_400/checkpoint-400", 
#         "checkpoint_freq": 20,
#         "checkpoint_at_end": True,
#         "max_failures": 999,
#         "stop": {
#             "training_iteration": 700,
#         },
#     },
# })